#XGBoost Sample - IBM Telco Customer Churn

Source : https://www.youtube.com/watch?v=GrJP9FLV3FE 
XGBoost in Python from Start to Finish




In [20]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix


In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
path = "gdrive/My Drive/tugas Machine Learning/"
df=pd.read_excel(path+'Telco_customer_churns.xlsx')
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [23]:
df.columns

Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value',
       'Churn Score', 'CLTV', 'Churn Reason'],
      dtype='object')

#Data Preprocessing


In [24]:
#drop unused features/columns for predictions
df.drop(['Churn Label', 'Churn Score','CLTV', 'Churn Reason'], axis=1, inplace=True) #axis=1 to remove columns
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Value
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,1
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,1


In [25]:

#check whether there are others unused columns, i.e single value
df['Count'].unique()

array([1])

In [26]:
df['Country'].unique


<bound method Series.unique of 0       United States
1       United States
2       United States
3       United States
4       United States
            ...      
7038    United States
7039    United States
7040    United States
7041    United States
7042    United States
Name: Country, Length: 7043, dtype: object>

In [27]:
df['Country'].unique()


array(['United States'], dtype=object)

In [28]:
df['State'].unique()

array(['California'], dtype=object)

In [29]:
df.drop(['CustomerID','Count','Country','State','Lat Long'], axis=1, inplace=True)
df.head()

,City,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Value
0,Los Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
1,Los Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1
2,Los Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,1
3,Los Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1
4,Los Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,1


In [30]:
df['City'].unique()

array(['Los Angeles', 'Beverly Hills', 'Huntington Park', ..., 'Standish',
       'Tulelake', 'Olympic Valley'], dtype=object)

In [31]:
# to draw tree, remove all whitespaces
df['City'].replace(' ','_', regex=True,inplace=True)
df.head()

,City,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Value
0,Los_Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
1,Los_Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1
2,Los_Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,1
3,Los_Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1
4,Los_Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,1


In [32]:
df['City'].unique()[0:10]

array(['Los_Angeles', 'Beverly_Hills', 'Huntington_Park', 'Lynwood',
       'Marina_Del_Rey', 'Inglewood', 'Santa_Monica', 'Torrance',
       'Whittier', 'La_Habra'], dtype=object)

In [33]:
#replace whitespace in the column names, requirement for drawing the tree

df.columns = df.columns.str.replace(' ','_')
df.head()

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,Multiple_Lines,Internet_Service,Online_Security,Online_Backup,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Value
0,Los_Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
1,Los_Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1
2,Los_Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,1
3,Los_Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1
4,Los_Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,1


#Identifying Missing Data #1

In [34]:
df.dtypes


City                  object
Zip_Code               int64
Latitude             float64
Longitude            float64
Gender                object
Senior_Citizen        object
Partner               object
Dependents            object
Tenure_Months          int64
Phone_Service         object
Multiple_Lines        object
Internet_Service      object
Online_Security       object
Online_Backup         object
Device_Protection     object
Tech_Support          object
Streaming_TV          object
Streaming_Movies      object
Contract              object
Paperless_Billing     object
Payment_Method        object
Monthly_Charges      float64
Total_Charges         object
Churn_Value            int64
dtype: object

In [35]:
df['Phone_Service'].unique()

array(['Yes', 'No'], dtype=object)

In [36]:
df['Total_Charges'].unique()

array([108.15, 151.65, 820.5, ..., 7362.9, 346.45, 6844.5], dtype=object)

In [37]:
df['Total_Charges'] = pd.to_numeric(df['Total_Charges'])

ValueError: ignored

#Dealing with Missing Data #1

In [ ]:
len(df.loc[df['Total_Charges']== ' '])

In [ ]:
df.loc[df['Total_Charges']== ' ']  #NULL, NA

In [ ]:
#perbaiki : opsi : delete atau ubah data jadi 0 
df.loc[(df['Total_Charges']== ' '), 'Total_Charges'] = 0

In [ ]:
#karena Tenure_Months is 0 ?

df.loc[df['Tenure_Months'] == 0]

In [ ]:

#Sudah tidak ada yang NULL/NA mari kita ubah tipe datanya

df['Total_Charges'] = pd.to_numeric(df['Total_Charges'])
df.dtypes

In [ ]:
#hilangkan semua whitespaces DI SEMUA DATA, SEMUA KOLOM agar bisa gambar
df.replace(' ','_', regex=True, inplace=True)
df.head()

In [ ]:
#Hmm...masih ada tipe data OBJECT ? sementara XGB only allows int, float or boolean


#Format Data #1 : Independent dan Dependent

In [ ]:
# buat data independent
X = df.drop('Churn_Value', axis=1).copy()
X.head()


In [ ]:
# buat data dependent
y = df['Churn_Value'].copy()
y.head()


#Format Data #2 : One-Hot-Encoding

In [ ]:
#XGB only support continuous data
#convert categorical data into multiple columns of binary value : One-Hot-Encoding
#terapkan untuk data categorical

df['Payment_Method'].unique()

In [ ]:
#coba terapkan 

pd.get_dummies(X, columns=['Payment_Method'])
X.head()

pd.get_dummies(X, columns=['Payment_Method']).head()

In [ ]:
#lakukan buat semua tipe OBJECT

X_encoded = pd.get_dummies(X,
	columns =['City',
		'Gender',
		'Senior_Citizen',
		'Partner',
		'Dependents',
		'Phone_Service',
		'Multiple_Lines',
		'Internet_Service',
		'Online_Security',
		'Online_Backup', 
		'Device_Protection', 
		'Tech_Support', 
		'Streaming_TV',
		'Streaming_Movies', 
		'Contract', 
		'Paperless_Billing', 
		'Payment_Method'])
X_encoded.head()

#Build A Preliminary XGB Model

In [ ]:
sum(y)/len(y)

In [ ]:
#Split the data agar proporsi yang left seimbang di train dan test

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=42, stratify=y)


In [ ]:
sum(y_train)/len(y_train)
sum(y_test)/len(y_test)
sum(y_train)/len(y_train), sum(y_test)/len(y_test)

In [ ]:
#create the model

clf_xgb = xgb.XGBClassifier(objective='binary:logistic', missing=None, seed=42)
clf_xgb.fit(X_train, 
	          y_train,
	          verbose=True, 
            early_stopping_rounds=10,
            eval_metric='aucpr',
	          eval_set=[(X_test, y_test)])


In [ ]:
#Check the accuracy
plot_confusion_matrix(clf_xgb,
			X_test, 
			y_test,
			values_format='d',
			display_labels=["Did not leave", "Left"])


In [ ]:
#1178/1294 = 91%
#239/467   = 51%

#Parameter Optimization

In [ ]:
#....singkat cerita, ketemu optimisasi paramnya

clf_xgb = xgb.XGBClassifier(seed=42,
			objective='binary:logistic',
			gamma=0.25,
			learn_rate=0.1,
			max_depth=4,
			reg_lambda=10,
			scale_pos_weight=3,
			subsample=0.9,
			colsample_bytree=0.5)

clf_xgb.fit(X_train, 
	          y_train,
	          verbose=True, 
            early_stopping_rounds=10,
            eval_metric='aucpr',
	          eval_set=[(X_test, y_test)])

In [ ]:
#Check the accuracy again
plot_confusion_matrix(clf_xgb,
			X_test, 
			y_test,
			values_format='d',
			display_labels=["Did not leave", "Left"])

In [ ]:

#left 	      
print(388/(388+79)*100)

In [ ]:
#did not leave
print(923/(923+371)*100)

#Draw the 1st tree

In [ ]:
#and let draw the tree !
clf_xgb = xgb.XGBClassifier(seed=42,
			objective='binary:logistic',
			gamma=0.25,
			learn_rate=0.1,
			max_depth=4,
			reg_lambda=10,
			scale_pos_weight=3,
			subsample=0.9,
			colsample_bytree=0.5,
			n_estimators=1) # We set this to 1 so we can get gain, cover, etc
clf_xgb.fit(X_train, y_train)



In [ ]:
bst = clf_xgb.get_booster()
for importance_type in ('weight', 'gain', 'cover', 'total_gain', 'total_cover'):
  print('%s: ' % importance_type, bst.get_score(importance_type=importance_type))

node_params = { 'shape' : 'box',
		'style' : 'filled, rounded',
		'fillcolor': '#78cbe'}
leaf_params = { 'shape' : 'box',
		'style' : 'filled, rounded',
		'fillcolor': '#e48038'}

xgb.to_graphviz(clf_xgb, num_trees=0, size="10,10",
		condition_node_params=node_params,
		leaf_node_params=leaf_params)
